In [1]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("test_set/test_set/cats/cat.4001.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)

1/1 [==============================] - 0s 337ms/step
Class: Cats
Confidence Score: 1.0


In [2]:
import os
from sklearn.metrics import classification_report

# Class names
class_names = ["cats", "dogs"]

# Paths
base_path = "test_set/test_set"

# True and predicted labels
y_true = []
y_pred = []

# Evaluate loop
for label_index, class_name in enumerate(class_names):
    folder = f"{base_path}/{class_name}"
    for file in os.listdir(folder):
        if file.lower().endswith('.jpg'):
            image_path = f"{folder}/{file}"
            image = Image.open(image_path).convert("RGB")
            image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
            image_array = np.asarray(image)
            normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
            data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
            data[0] = normalized_image_array

            prediction = model.predict(data, verbose=0)
            predicted_index = np.argmax(prediction)

            y_true.append(label_index)
            y_pred.append(predicted_index)

# Show evaluation
print(classification_report(y_true, y_pred, target_names=class_names))

              precision    recall  f1-score   support

        cats       0.97      0.97      0.97      1011
        dogs       0.97      0.97      0.97      1012

    accuracy                           0.97      2023
   macro avg       0.97      0.97      0.97      2023
weighted avg       0.97      0.97      0.97      2023

